# Data loading

# Python packages




In [5]:
import math, datetime, time, random
import numpy as np
import pandas as pd
from datetime import timedelta
import re
import math, datetime, time, random
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


# Door open - inbound




In [6]:
inbound = pd.read_csv('inbound_loads.csv')
ft = pd.read_csv('featuretrain.csv')

### Uitleg

1. Get all the minutes for the interval in between start and finish


In [7]:
def interval_min(start_date, end_date):
    dates_range = []
    while start_date <= end_date:
        dates_range.append(start_date) # or you can append str(start_date) if want the dates to be as strings
        start_date += timedelta(minutes=1)
    return dates_range

In [8]:
door = inbound.filter(['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], axis=1)
door.head(5)

,load_start_datetime,load_finish_datetime,dock_door_number
0,2019-02-11 04:42:00,2019-02-11 04:51:00,59.0
1,2019-01-16 10:11:00,2019-01-16 11:27:00,60.0
2,2019-07-30 09:59:00,2019-07-30 10:18:00,61.0
3,2019-01-10 08:51:00,2019-01-10 08:58:00,58.0
4,2019-01-10 07:26:00,2019-01-10 08:08:00,56.0


2. Get all the minutes intervals and put them in a new dataframe

In [9]:
door['load_finish_datetime'] = pd.to_datetime(door['load_finish_datetime'])
door['load_start_datetime'] = pd.to_datetime(door['load_start_datetime'])

door['load_minutes'] = np.nan

for index, row in door.iterrows():
    door['load_minutes'].iloc[index] = interval_min(door['load_start_datetime'].iloc[index], door['load_finish_datetime'].iloc[index])

C:\Users\suzan\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
door = door.drop_duplicates(subset=['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], keep="first")

3. Make a flat list from the 'load_minutes' column 

In [11]:
def flatten(l):
    return [item for sublist in l for item in sublist]

list_min = door['load_minutes'].tolist()
list_min = flatten(list_min)
type(list_min[0])

pandas._libs.tslibs.timestamps.Timestamp

4. Create new dataframe with each time as an individual row (instead of list in each row). And change type from timeFrame to Datetime

In [13]:
minutes = pd.DataFrame(list_min)
minutes.columns =['datetime_local']
#minutes['datetime_local'] = pd.to_datetime(minutes['datetime_local'], unit='s')
#minutes['datetime_local'] = pd.Series(minutes.datetime_local.dt.to_pydatetime(), dtype='O')

5. Create a new dataframe with datetime and count

In [14]:
test = minutes.value_counts().to_frame('counts')
#test = minutes.value_counts().rename_axis('datetime_local').to_frame('counts')
test

,counts
datetime_local,
2021-12-23 09:48:00,12
2021-12-23 09:50:00,12
2021-12-23 09:51:00,12
2021-12-23 09:52:00,12
2021-12-23 09:53:00,12
...,...
2020-04-10 17:55:00,1
2020-04-10 17:56:00,1
2020-04-10 17:57:00,1


Bit sketchy but works.
- Reset the index level of dataframe 'test' and then turn Timeframe type into datetime
- Do the same with dataframe 'ft', so reset index level (is string), then turn type to datetime.


In [15]:
test = test.reset_index(level=[0])
#type(test.index)
#type(test['datetime_local'][0])

test['datetime_local'] = pd.to_datetime(test['datetime_local'], unit='s')
test['datetime_local'] = pd.Series(test.datetime_local.dt.to_pydatetime(), dtype='O')

In [31]:
ft = ft.reset_index(level=[0])
type(ft['datetime_UTC'][0])

str

In [32]:
ft['datetime_local'] = pd.to_datetime(ft['datetime_local'], unit='s')
ft['datetime_local'] = pd.Series(ft.datetime_local.dt.to_pydatetime(), dtype='O')
type(ft.datetime_local[0])

datetime.datetime

- Set datetime_local as index again. (Now the type of both is the same, which is necessary for merging correctly).

In [33]:
ft.set_index('datetime_local', inplace=True)
test.set_index('datetime_local', inplace=True)

KeyError: "None of ['datetime_local'] are in the columns"

- Now merge ft and test dataframe, on the content of the datetime_local of the ft dataframe. 


In [34]:
feat_min = ft.merge(test, left_on='datetime_local', right_index=True, how='left').reset_index()

In [38]:
feat_min["counts"] = feat_min["counts"].fillna(0)

#Drop certain columns if necessary
#feat_min = feat_min.drop(['level_0_x', 'index_x','level_0_y', 'index_y'], axis=1)
feat_min
#feat_min.to_csv("feat_door_open")

,level_0,index,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,counts
datetime_local,,,,,,,,,,,,,
2018-12-31 18:00:00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:00:00-06:00,50.37,53.6,2019-01-01 00:00:00,1.0
2018-12-31 18:05:00,1,1,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:05:00-06:00,50.37,53.6,2019-01-01 00:05:00,1.0
2018-12-31 18:10:00,2,2,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:10:00-06:00,50.37,53.6,2019-01-01 00:10:00,1.0
2018-12-31 18:15:00,3,3,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:15:00-06:00,50.37,53.6,2019-01-01 00:15:00,0.0
2018-12-31 18:20:00,4,4,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:20:00-06:00,50.37,53.6,2019-01-01 00:20:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 17:35:00,485205,485205,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31 17:35:00-06:00,82.89,68.0,2021-12-31 23:35:00,0.0
2021-12-31 17:40:00,485206,485206,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31 17:40:00-06:00,82.89,68.0,2021-12-31 23:40:00,0.0
2021-12-31 17:45:00,485207,485207,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31 17:45:00-06:00,82.89,68.0,2021-12-31 23:45:00,0.0


In [49]:
#feat_min.set_index('datetime_local', inplace=True)
#feat_min = feat_min.drop(['level_0', 'index','year', 'month','weekday', 'day', 'hour', 'minute', 'datetime'], axis=1)
#feat_min = feat_min.drop(['Temperature', 'Relative Humidity', 'datetime_UTC'], axis=1)
feat_min
feat_min.to_csv("feature_train_inbound_door_open")

In [ ]:
feat_min.iloc[400]